In [1]:
import numpy as np
import os
import time
import datetime as dt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, datasets
from torch.autograd import Variable
import torchvision

In [2]:
# Send the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Define transforms to be applied to the image data
transform = transforms.Compose([
    transforms.Resize(64),
    #transforms.CenterCrop(100),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
# Load the dataset
train_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_bone/bone_train/", transform=transform)

# Define the data loader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Define the label names
label_names = train_dataset.classes

In [5]:
# Load the dataset
val_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_bone/bone_test/", transform=transform)

# Define the data loader
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

# Define the label names
label_names = val_dataset.classes

In [6]:
print(label_names)

['hand_A', 'hand_A2', 'hand_B', 'hand_C', 'hand_D', 'hand_D2', 'hand_E', 'hand_G', 'hand_H', 'hand_I', 'hand_K', 'hand_L', 'hand_M', 'hand_N', 'hand_O', 'hand_O3', 'hand_P', 'hand_Q', 'hand_R', 'hand_S', 'hand_T', 'hand_U', 'hand_V', 'hand_X', 'hand_Y']


In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.fc = nn.Linear(128 * 8 * 8, 25)  # fully-connected layer for classification

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(-1, 128 * 8 * 8)  # flatten the output of the last convolutional layer
        x = self.fc(x)
        return x

In [8]:
model = ConvNet().to(device)
print(model)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=8192, out_features=25, bias=True)
)


In [9]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1
start_time = time.time()
# Loop over the dataset and train the model
model.train()
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()
        
        #send data to cuda
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Print training statistics
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train time: {:.2f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), (time.time()-start_time)/60))
'''
    # Evaluate the model on the validation dataset
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy on validation set: {:.2f}%'.format(100 * correct / total))
'''


Epoch [1/1], Step [100/273716], Loss: 1.3975, Train time: 0.14
Epoch [1/1], Step [200/273716], Loss: 0.0238, Train time: 0.17
Epoch [1/1], Step [300/273716], Loss: 0.1436, Train time: 0.21
Epoch [1/1], Step [400/273716], Loss: 1.3123, Train time: 0.24
Epoch [1/1], Step [500/273716], Loss: 0.1789, Train time: 0.27
Epoch [1/1], Step [600/273716], Loss: 1.7096, Train time: 0.34
Epoch [1/1], Step [700/273716], Loss: 0.1510, Train time: 0.39
Epoch [1/1], Step [800/273716], Loss: 0.0326, Train time: 0.43
Epoch [1/1], Step [900/273716], Loss: 0.1464, Train time: 0.49
Epoch [1/1], Step [1000/273716], Loss: 0.4760, Train time: 0.53
Epoch [1/1], Step [1100/273716], Loss: 0.1601, Train time: 0.56
Epoch [1/1], Step [1200/273716], Loss: 0.0168, Train time: 0.59
Epoch [1/1], Step [1300/273716], Loss: 0.0005, Train time: 0.64
Epoch [1/1], Step [1400/273716], Loss: 0.0015, Train time: 0.70
Epoch [1/1], Step [1500/273716], Loss: 1.5158, Train time: 0.73
Epoch [1/1], Step [1600/273716], Loss: 0.1540, Tr

OSError: [Errno 22] Invalid argument

In [10]:
# Evaluate the model on the validation dataset
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # Print training statistics
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Accuracy: {:.2f}, Test time: {:.2f}'
                      .format(epoch+1, num_epochs, i+1, len(val_loader), 100 * correct / total, (time.time()-start_time)/60))

    print('Accuracy on test set: {:.2f}%'.format(100 * correct / total))

Epoch [1/1], Step [100/89295], Accuracy: 89.00, Test time: 0.12
Epoch [1/1], Step [200/89295], Accuracy: 89.50, Test time: 0.20
Epoch [1/1], Step [300/89295], Accuracy: 88.67, Test time: 0.28
Epoch [1/1], Step [400/89295], Accuracy: 88.25, Test time: 0.36
Epoch [1/1], Step [500/89295], Accuracy: 89.20, Test time: 0.43
Epoch [1/1], Step [600/89295], Accuracy: 89.50, Test time: 0.50
Epoch [1/1], Step [700/89295], Accuracy: 89.71, Test time: 0.58
Epoch [1/1], Step [800/89295], Accuracy: 90.00, Test time: 0.65
Epoch [1/1], Step [900/89295], Accuracy: 89.89, Test time: 0.73
Epoch [1/1], Step [1000/89295], Accuracy: 89.70, Test time: 0.81
Epoch [1/1], Step [1100/89295], Accuracy: 89.82, Test time: 0.88
Epoch [1/1], Step [1200/89295], Accuracy: 89.67, Test time: 0.98
Epoch [1/1], Step [1300/89295], Accuracy: 90.00, Test time: 1.06
Epoch [1/1], Step [1400/89295], Accuracy: 89.71, Test time: 1.13
Epoch [1/1], Step [1500/89295], Accuracy: 89.80, Test time: 1.23
Epoch [1/1], Step [1600/89295], Ac